# Consultas descriptivas del modelo

## Importación de librerías necesarias

In [1]:
import pandas as pd

## Importación de bases de datos

In [2]:
# Games
games = pd.read_parquet("../Data/final/games_final.parquet")
# Reviews
reviews = pd.read_parquet("../Data/final/reviews_final.parquet")
# Items
items = pd.read_parquet("../Data/final/items_final.parquet")

## Creación de consultas

### PlayTimeGenre

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [84]:
# Se define la función PlayTimeGenre
def PlayTimeGenre(genero:str):
    try:
        # Se obtiene la base de juegos y se extraen los géneros por juego a través de un explode.
        g = games.explode("genres")
        # Se obtienen únicamente los juegos del género especificado.
        g = g[g["genres"] == genero]
        # Se obtiene la sumatoria de horas jugadas por juego a partir del dataframe de items.
        i = items.groupby("item_id")["playtime_forever"].sum()
        # Se obtiene un dataframe con las horas por juego y el año de lanzamiento.
        s = pd.merge(g,i,how = "inner",left_on="id",right_on="item_id")
        # Se agrupan las horas jugadas por año de lanzamiento.
        s = s.groupby(s["release_date"].dt.year)["playtime_forever"].sum()

        # se obtiene el año con más horas jugadas.
        año = s.idxmax()
        
        # Se genera la respuesta.
        respuesta = {f"Año de lanzamiento con más horas jugadas para el género {genero}":año}

        return respuesta
    # Se crea una excepción para los casos en donde el género señalado no esté en la base de datos.
    except Exception:
        return {"No se encontraron datos para el género referenciado"}


In [4]:
# Se prueba la fórmula
genero = "Action"
PlayTimeGenre(genero)

{'Año de lanzamiento con más horas jugadas para el género Action': 2012}

In [5]:
# Se prueba la fórmula
genero = "Free to Play"
PlayTimeGenre(genero)

{'Año de lanzamiento con más horas jugadas para el género Free to Play': 2013}

In [6]:
# Se prueba la fórmula
genero = "RPG"
PlayTimeGenre(genero)

{'Año de lanzamiento con más horas jugadas para el género RPG': 2011}

### UserForGenre

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [83]:
# Se define la función UserForGenre
def UserForGenre(genero:str):
    try:
        # Se obtiene la base de juegos y se extraen los géneros por juego a través de un explode.
        g = games.explode("genres")
        # Se obtienen únicamente los juegos del género especificado.
        g = g[g["genres"] == genero][["release_date","id"]]
        # Se obtiene la lista de id de juegos del género especificado.
        juegos = list(g["id"].unique())
        # Se obtienen los juegos de los usuarios que cumplen con la condición del género.
        i = items[items["item_id"].isin(juegos)]
        # Se unen en un mismo dataframe el año de lanzamiento y las horas jugadas. Se decide usar el año de lanzamiento como referencia para los juegos utilizados. Si bien esto no es garantía de que efectivamente haya jugado el juego en esta fecha, nos sirve como referencia más amplia de años que los datos en review.
        i = pd.merge(i,g,how="inner",left_on="item_id",right_on="id")
        # Se obtienen la sumatoria de horas por usuario.
        i = i.groupby(["user_id",i["release_date"].dt.year])["playtime_forever"].sum()
        # Se obtiene el id del jugador con más horas.
        player = i.idxmax()[0]
        # Se obtiene la información para el jugador identificado.
        player_data = i.loc[player]

        # Se estructura la respuesta: 
        response_data = {
            f"Usuario con más horas jugadas para el género {genero}":player,
            "Horas jugadas": [{"año":year,
                            "horas":playtime
                            }
                            for year,playtime in player_data.items()
                            ]
        }

        return response_data
    # Se crea una excepción para los casos en donde el género señalado no esté en la base de datos.
    except Exception:
        return {"No se encontraron datos para el género referenciado"}

In [33]:
# Se prueba la función.
genero = "Action"
UserForGenre(genero)

{'Usuario con más horas jugadas para el género Action': 'shinomegami',
 'Horas jugadas': [{'año': 1988, 'horas': 2.27},
  {'año': 1995, 'horas': 0.03},
  {'año': 1996, 'horas': 19.35},
  {'año': 1997, 'horas': 1.65},
  {'año': 1998, 'horas': 61.93},
  {'año': 1999, 'horas': 5.06},
  {'año': 2000, 'horas': 179.59},
  {'año': 2001, 'horas': 40.49},
  {'año': 2003, 'horas': 8879.26},
  {'año': 2004, 'horas': 20.42},
  {'año': 2005, 'horas': 5.71},
  {'año': 2006, 'horas': 36.339999999999996},
  {'año': 2007, 'horas': 1140.42},
  {'año': 2008, 'horas': 3.0},
  {'año': 2009, 'horas': 3553.7599999999998},
  {'año': 2010, 'horas': 159.8},
  {'año': 2011, 'horas': 2611.6400000000003},
  {'año': 2012, 'horas': 346.39},
  {'año': 2013, 'horas': 2772.99},
  {'año': 2014, 'horas': 3013.1499999999996},
  {'año': 2015, 'horas': 2772.44},
  {'año': 2016, 'horas': 712.5},
  {'año': 2017, 'horas': 2.25}]}

In [34]:
# Se prueba la función.
genero = "Free to Play"
UserForGenre(genero)

{'Usuario con más horas jugadas para el género Free to Play': 'idonothack',
 'Horas jugadas': [{'año': 2006, 'horas': 0.3},
  {'año': 2011, 'horas': 0.13},
  {'año': 2012, 'horas': 29.9},
  {'año': 2013, 'horas': 498.57},
  {'año': 2014, 'horas': 16.57},
  {'año': 2015, 'horas': 12617.07},
  {'año': 2016, 'horas': 269.43},
  {'año': 2017, 'horas': 38.7}]}

In [35]:
# Se prueba la función.
genero = "RPG"
UserForGenre(genero)

{'Usuario con más horas jugadas para el género RPG': 'shinomegami',
 'Horas jugadas': [{'año': 1999, 'horas': 6.289999999999999},
  {'año': 2003, 'horas': 8849.050000000001},
  {'año': 2004, 'horas': 11.6},
  {'año': 2005, 'horas': 3.08},
  {'año': 2006, 'horas': 33.72},
  {'año': 2007, 'horas': 2250.34},
  {'año': 2008, 'horas': 21.75},
  {'año': 2009, 'horas': 23.03},
  {'año': 2010, 'horas': 690.12},
  {'año': 2011, 'horas': 295.89},
  {'año': 2012, 'horas': 213.64},
  {'año': 2013, 'horas': 1796.2299999999998},
  {'año': 2014, 'horas': 1365.6499999999999},
  {'año': 2015, 'horas': 1856.97},
  {'año': 2016, 'horas': 259.17}]}

### UsersRecommend
def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [78]:
# Se define la función UsersRecommend
def UsersRecommend(año:int):
    try:
        # Se obtienen todos los juegos que tuvieron recomendaciones en el año especificado y cuentan con una recomendación positiva.
        r = reviews[(reviews["posted"].dt.year == año) & (reviews["recommend"] == True) & (reviews["sentiment_analysis"].isin([1,2]))][["item_id","posted","sentiment_analysis"]]
        # Se obtiene la sumatoria de favorabilidad del juego.
        h = r.groupby("item_id")["sentiment_analysis"].sum()
        # Se obtienen los tres juegos más recomendados.
        recomendados = h.nlargest(3).reset_index()

        # Se estructura la respuesta. 
        respuesta = {
            "Puesto 1": games.loc[games["id"] == recomendados.loc[0,"item_id"],"app_name"].iloc[0],
            "Puesto 2": games.loc[games["id"] == recomendados.loc[1,"item_id"],"app_name"].iloc[0],
            "Puesto 3": games.loc[games["id"] == recomendados.loc[2,"item_id"],"app_name"].iloc[0],
        }

        return respuesta
    
    # Se crea una excepción para los casos en donde el año señalado no esté en la base de datos.
    except Exception as e:
        return {"No se encontraron datos para el año indicado"}

In [76]:
# Se prueba la función
año = 2015
UsersRecommend(año)

{'Puesto 1': 'Counter-Strike: Global Offensive',
 'Puesto 2': 'Team Fortress 2',
 'Puesto 3': "Garry's Mod"}

In [79]:
# Se prueba la función
año = 2016
UsersRecommend(año)

{'No se encontraron datos para el año indicado'}

In [80]:
# Se prueba la función
año = 2013
UsersRecommend(año)

{'Puesto 1': 'Team Fortress 2',
 'Puesto 2': "Garry's Mod",
 'Puesto 3': 'Left 4 Dead 2'}

In [81]:
# Se prueba la función
año = 2014
UsersRecommend(año)

{'Puesto 1': 'Team Fortress 2',
 'Puesto 2': 'Counter-Strike: Global Offensive',
 'Puesto 3': "Garry's Mod"}

### UsersNotRecommend
def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [88]:
# Se define la función UsersNotRecommend
def UsersNotRecommend(año:int):
    try:
        # Se obtienen todos los juegos que tuvieron recomendaciones en el año especificado y cuentan con una recomendación negativa.
        r = reviews[(reviews["posted"].dt.year == año) & (reviews["recommend"] == False) & (reviews["sentiment_analysis"] == 0)][["item_id","posted","sentiment_analysis"]]
        # Se obtiene el número de recomendaciones negativas del juego.
        h = r.groupby("item_id")["sentiment_analysis"].count()
        # Se obtienen los tres juegos menos recomendados.
        recomendados = h.nlargest(3).reset_index()

        # Se estructura la respuesta. 
        respuesta = {
            "Puesto 1": games.loc[games["id"] == recomendados.loc[0,"item_id"],"app_name"].iloc[0],
            "Puesto 2": games.loc[games["id"] == recomendados.loc[1,"item_id"],"app_name"].iloc[0],
            "Puesto 3": games.loc[games["id"] == recomendados.loc[2,"item_id"],"app_name"].iloc[0],
        }

        return respuesta
    
    # Se crea una excepción para los casos en donde el año señalado no esté en la base de datos.
    except Exception:
        return {"No se encontraron datos para el año especificado"}

In [89]:
# Se prueba la función
año = 2015
UsersNotRecommend(año)

{'Puesto 1': 'Counter-Strike: Global Offensive',
 'Puesto 2': 'Rust',
 'Puesto 3': 'DayZ'}

In [90]:
# Se prueba la función
año = 2011
UsersNotRecommend(año)

{'Puesto 1': 'And Yet It Moves',
 'Puesto 2': 'From Dust',
 'Puesto 3': 'Men of War: Vietnam'}

In [91]:
# Se prueba la función
año = 2017
UsersNotRecommend(año)

{'No se encontraron datos para el año especificado'}

### sentiment_analysis
def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [92]:
# Se define la función sentiment_analysis
def sentiment_analysis(año:int):
    try:
        # Se obtienen todas las reseñas para el año especificado:
        r = reviews[reviews["posted"].dt.year == año]
        # Se obtienen el número de comentario por cada categoria
        r = r.groupby("sentiment_analysis")["user_id"].count()
        # Se estructura la respuesta
        respuesta = {
        "Negative":r[0],
        "Neutral":r[1],
        "Positive":r[2]
        }

        return respuesta
    # Se crea una excepción para los casos en donde el año señalado no esté en la base de datos.
    except Exception:
        return {"No existen datos para el año referenciado"}

In [93]:
# Se prueba la funcion
año = 2014
sentiment_analysis(año)

{'Negative': 1012, 'Neutral': 4168, 'Positive': 8334}

In [94]:
# Se prueba la funcion
año = 2015
sentiment_analysis(año)

{'Negative': 1104, 'Neutral': 4113, 'Positive': 6639}

In [95]:
# Se prueba la funcion
año = 2013
sentiment_analysis(año)

{'Negative': 234, 'Neutral': 1205, 'Positive': 2670}

In [96]:
# Se prueba la funcion
año = 2018
sentiment_analysis(año)

{'No existen datos para el año referenciado'}